In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from xgboost import XGBRegressor

In [2]:
train_data = pd.read_csv(r'C:\Users\1449415\Analytics\Projects\AV_hackathon\Data\TRAIN.csv', parse_dates=['Date'])
test_data = pd.read_csv(r'C:\Users\1449415\Analytics\Projects\AV_hackathon\Data\TEST_FINAL.csv', parse_dates=['Date'])

In [3]:
train_data.head()#2018-01-01,2019-05-31

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [4]:
test_data.head()#2019-06-01,2019-07-31

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No


### Max and Min Dates

In [28]:
train_data['Date'].min(),train_data['Date'].max() # roughly 1.5 year data(daily data)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2019-05-31 00:00:00'))

In [29]:
test_data['Date'].min(),test_data['Date'].max() # to predict 2 months data(daily data)

(Timestamp('2019-06-01 00:00:00'), Timestamp('2019-07-31 00:00:00'))

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             188340 non-null  object        
 1   Store_id       188340 non-null  int64         
 2   Store_Type     188340 non-null  object        
 3   Location_Type  188340 non-null  object        
 4   Region_Code    188340 non-null  object        
 5   Date           188340 non-null  datetime64[ns]
 6   Holiday        188340 non-null  int64         
 7   Discount       188340 non-null  object        
 8   #Order         188340 non-null  int64         
 9   Sales          188340 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 14.4+ MB


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22265 entries, 0 to 22264
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             22265 non-null  object        
 1   Store_id       22265 non-null  int64         
 2   Store_Type     22265 non-null  object        
 3   Location_Type  22265 non-null  object        
 4   Region_Code    22265 non-null  object        
 5   Date           22265 non-null  datetime64[ns]
 6   Holiday        22265 non-null  int64         
 7   Discount       22265 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 1.4+ MB


### Missing Values

In [8]:
train_data.isna().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

In [9]:
test_data.isna().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
dtype: int64

#### Inference: No missing value treatment required

In [13]:
lis = ['Store_Type','Location_Type','Region_Code']
for i in lis:
    print(i)
    print(train_data[i].value_counts())

Store_Type
S1    88752
S4    45924
S2    28896
S3    24768
Name: Store_Type, dtype: int64
Location_Type
L1    85140
L2    48504
L3    29928
L5    13932
L4    10836
Name: Location_Type, dtype: int64
Region_Code
R1    63984
R2    54180
R3    44376
R4    25800
Name: Region_Code, dtype: int64


In [17]:
train_data.groupby('Store_Type')['Sales'].mean().sort_values(ascending=False)

Store_Type
S4    59945.685926
S3    47063.068209
S1    37676.511694
S2    27530.828222
Name: Sales, dtype: float64

In [20]:
train_data.groupby('Location_Type')['Sales'].mean().sort_values(ascending=False)

Location_Type
L2    59231.480373
L1    41453.597889
L3    33072.257756
L4    29067.414313
L5    25187.787261
Name: Sales, dtype: float64

In [21]:
train_data.groupby('Region_Code')['Sales'].mean().sort_values(ascending=False)

Region_Code
R1    46765.488405
R3    42144.517063
R2    40054.847344
R4    39743.434249
Name: Sales, dtype: float64

In [22]:
#Label Encoding Store columns - Train data
mapper_store_type = {'S1':3,'S2':4,'S3':2,'S4':1}
mapper_location_type = {'L1':4,'L2':5,'L3':3,'L4':2,'L5':1}
mapper_region_code = {'R1':4,'R2':2,'R3':3,'R4':1}
train_data['Store_Type'] = train_data['Store_Type'].map(mapper_store_type)
train_data['Location_Type'] = train_data['Location_Type'].map(mapper_location_type)
train_data['Region_Code'] = train_data['Region_Code'].map(mapper_region_code)

In [23]:
#Label Encoding Store columns - Test data
test_data['Store_Type'] = test_data['Store_Type'].map(mapper_store_type)
test_data['Location_Type'] = test_data['Location_Type'].map(mapper_location_type)
test_data['Region_Code'] = test_data['Region_Code'].map(mapper_region_code)

In [25]:
train_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,3,3,4,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,1,5,4,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,2,5,4,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,4,3,4,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,4,3,1,2018-01-01,1,Yes,62,45614.52


In [26]:
test_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,1,5,3,2019-06-01,0,No
1,T1188342,172,3,4,4,2019-06-01,0,No
2,T1188343,173,1,5,4,2019-06-01,0,No
3,T1188344,174,3,4,1,2019-06-01,0,No
4,T1188345,170,3,4,2,2019-06-01,0,No


### Make dates as indexes

In [31]:
train_data.set_index('Date',inplace=True)
test_data.set_index('Date',inplace=True)

In [32]:
train_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount,#Order,Sales
Date,,,,,,,,,
2018-01-01,T1000001,1,3,3,4,1,Yes,9,7011.84
2018-01-01,T1000002,253,1,5,4,1,Yes,60,51789.12
2018-01-01,T1000003,252,2,5,4,1,Yes,42,36868.20
2018-01-01,T1000004,251,4,3,4,1,Yes,23,19715.16
2018-01-01,T1000005,250,4,3,1,1,Yes,62,45614.52


In [33]:
test_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount
Date,,,,,,,
2019-06-01,T1188341,171,1,5,3,0,No
2019-06-01,T1188342,172,3,4,4,0,No
2019-06-01,T1188343,173,1,5,4,0,No
2019-06-01,T1188344,174,3,4,1,0,No
2019-06-01,T1188345,170,3,4,2,0,No


In [52]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 188340 entries, 2018-01-01 to 2019-05-31
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  int64  
 3   Location_Type  188340 non-null  int64  
 4   Region_Code    188340 non-null  int64  
 5   Holiday        188340 non-null  int64  
 6   Discount       188340 non-null  object 
 7   #Order         188340 non-null  int64  
 8   Sales          188340 non-null  float64
dtypes: float64(1), int64(6), object(2)
memory usage: 18.4+ MB


In [67]:
## Discount is still categorical let's do One Hot Encoding for it using pandas get_dummies()
train_data=pd.get_dummies(train_data,columns=['Discount'],drop_first=True)
test_data=pd.get_dummies(test_data,columns=['Discount'],drop_first=True)

In [54]:
train_data.columns

Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code',
       'Holiday', '#Order', 'Sales', 'Discount_No', 'Discount_Yes'],
      dtype='object')

In [70]:
train_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,#Order,Sales,Discount_Yes
Date,,,,,,,,,
2018-01-01,T1000001,1,3,3,4,1,9,7011.84,1
2018-01-01,T1000002,253,1,5,4,1,60,51789.12,1
2018-01-01,T1000003,252,2,5,4,1,42,36868.20,1
2018-01-01,T1000004,251,4,3,4,1,23,19715.16,1
2018-01-01,T1000005,250,4,3,1,1,62,45614.52,1


In [71]:
test_data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount_Yes
Date,,,,,,,
2019-06-01,T1188341,171,1,5,3,0,0
2019-06-01,T1188342,172,3,4,4,0,0
2019-06-01,T1188343,173,1,5,4,0,0
2019-06-01,T1188344,174,3,4,1,0,0
2019-06-01,T1188345,170,3,4,2,0,0


In [72]:
X_train = train_data[:'2019-03-01 00:00:00'].drop(columns=['ID','#Order','Sales'])
Y_train = train_data['Sales'][:'2019-03-01 00:00:00']
X_val = train_data['2019-03-01 00:00:00':].drop(columns=['ID','#Order','Sales'])
Y_val = train_data['Sales']['2019-03-01 00:00:00':]

In [73]:
X_train.index.max(),X_train.index.min()

(Timestamp('2019-03-01 00:00:00'), Timestamp('2018-01-01 00:00:00'))

In [74]:
X_val.index.max(),X_val.index.min()

(Timestamp('2019-05-31 00:00:00'), Timestamp('2019-03-01 00:00:00'))

In [75]:
X_train.columns

Index(['Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Holiday',
       'Discount_Yes'],
      dtype='object')

### XGBoost Training

In [80]:
xgb = XGBRegressor(n_estimators = 1000, objective='reg:linear')

In [81]:
xgb.fit(X_train,Y_train,eval_set=[(X_train,Y_train),(X_val,Y_val)],early_stopping_rounds=50,verbose=True)

[22:56:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:33438.21094	validation_1-rmse:35685.08594
[1]	validation_0-rmse:24854.67773	validation_1-rmse:27390.40234
[2]	validation_0-rmse:19287.74023	validation_1-rmse:22036.78711
[3]	validation_0-rmse:15763.37598	validation_1-rmse:18628.84570
[4]	validation_0-rmse:13702.67481	validation_1-rmse:16610.39648
[5]	validation_0-rmse:12490.86426	validation_1-rmse:15360.26074
[6]	validation_0-rmse:11832.15137	validation_1-rmse:14641.51269
[7]	validation_0-rmse:11393.06348	validation_1-rmse:14146.65820
[8]	validation_0-rmse:11167.28516	validation_1-rmse:13884.40039
[9]	validation_0-rmse:11057.28027	validation_1-rmse:13727.93750
[10]	validation_0-rmse:10970.16504	validation_1-rmse:13598.69727
[11]	validation_0-rmse:10877.75879	validation_1-rmse:13494.68457
[12]	validation_0-rmse:10830.09863	validation_1-rmse:

[124]	validation_0-rmse:10305.62598	validation_1-rmse:13024.18652
[125]	validation_0-rmse:10305.22168	validation_1-rmse:13024.09375
[126]	validation_0-rmse:10304.99707	validation_1-rmse:13024.06250
[127]	validation_0-rmse:10304.39941	validation_1-rmse:13023.99805
[128]	validation_0-rmse:10304.31738	validation_1-rmse:13023.98926
[129]	validation_0-rmse:10304.25488	validation_1-rmse:13024.04981
[130]	validation_0-rmse:10304.15820	validation_1-rmse:13024.11231
[131]	validation_0-rmse:10303.87109	validation_1-rmse:13024.12207
[132]	validation_0-rmse:10303.71191	validation_1-rmse:13024.65820
[133]	validation_0-rmse:10303.49414	validation_1-rmse:13024.97070
[134]	validation_0-rmse:10302.97559	validation_1-rmse:13024.54492
[135]	validation_0-rmse:10302.56543	validation_1-rmse:13024.86328
[136]	validation_0-rmse:10302.28906	validation_1-rmse:13024.85742
[137]	validation_0-rmse:10302.17188	validation_1-rmse:13024.87695
[138]	validation_0-rmse:10301.96289	validation_1-rmse:13024.92383
[139]	vali

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

### Prediction

In [82]:
pred_val_res = xgb.predict(X_val)
pred_val_res

array([43782.01 , 69093.82 , 50222.816, ..., 25204.87 , 40596.742,
       24856.074], dtype=float32)

In [83]:
## Evaluation using Mean Squared Log Error
mean_squared_log_error(Y_val,pred_val_res)

0.11875210935319848

In [87]:
pred_test = xgb.predict(test_data.drop(columns='ID'))

In [88]:
pred_test

array([47994.652, 36201.727, 70782.586, ..., 47936.2  , 32081.922,
       24893.174], dtype=float32)

In [92]:
test_data

,ID,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount_Yes
Date,,,,,,,
2019-06-01,T1188341,171,1,5,3,0,0
2019-06-01,T1188342,172,3,4,4,0,0
2019-06-01,T1188343,173,1,5,4,0,0
2019-06-01,T1188344,174,3,4,1,0,0
2019-06-01,T1188345,170,3,4,2,0,0
...,...,...,...,...,...,...,...
2019-07-31,T1210601,186,4,1,2,0,0
2019-07-31,T1210602,11,1,5,4,0,0
2019-07-31,T1210603,185,3,4,3,0,1


### Preparing Submissions

In [89]:
submission = pd.read_csv(r'C:\Users\1449415\Analytics\Projects\AV_hackathon\Data\Sample.csv')

In [90]:
submission.head()

,ID,Sales
0,T1188341,42275
1,T1188342,42275
2,T1188343,42275
3,T1188344,42275
4,T1188345,42275


In [93]:
submission['Sales'] = pred_test#we've predicted in the same order as required

In [94]:
submission.to_csv(r'C:\Users\1449415\Analytics\Projects\AV_hackathon\Data\xgb_submission.csv',index=False)